# Stage 3: Blocking

### Import Required Packages

In [1]:
from urllib2 import urlopen
import pandas as pd
import re
import numpy as np
import nltk
from collections import defaultdict
from datetime import datetime

### Load & Prepare Tables
1. __Load table data from Github__
    1. Spoj Data = *table_A*
    2. Codechef Data = *table_B*
2. __Combine Text Attributes__
    1. Concatenate _description_ + _input_ + _output_ text attributes into single _words_ attribute
    2. Drop _description_ + _input_ + _output_ text attributes

In [2]:
table_spoj = "https://raw.githubusercontent.com/KaranTalreja/CS638/master/spoj/json/spoj_blocking.csv?token=AKuoLTvhQla5FVgd-sWedBmZ7C9MitFsks5YLxDGwA%3D%3D"
table_A = pd.read_csv(urlopen(table_spoj)).drop("Unnamed: 0", axis=1)
table_A["words"] = table_A["description"].fillna("") + table_A["input"].fillna("") + table_A["output"].fillna("")
table_A = table_A.drop(["description", "input", "output"], axis=1)

In [3]:
table_codechef = "https://raw.githubusercontent.com/KaranTalreja/CS638/master/codechef/data/codechef_blocking.csv?token=AD_cZbkPhGiqHVai0GuHnKYh2i7k3xIlks5YMLsuwA%3D%3D"
table_B = pd.read_csv(urlopen(table_codechef))
table_B["words"] = table_B["description"].fillna("") + table_B["input"].fillna("") + table_B["output"].fillna("")
table_B = table_B.drop(["description", "input", "output"], axis=1)

### Inverted Index based on solve_rate for both tables

In [4]:
inverse_indexing_rate_A = defaultdict(list)
bins = [x/2.0 for x in range(0,10,2)]
for index, value in table_A["solve_rate_normalized"].iteritems():
    for upper_bound in bins:
        if value < upper_bound:
            inverse_indexing_rate_A[upper_bound].append(index)
            break

In [5]:
inverse_indexing_rate_B = defaultdict(list)
bins = [x/2.0 for x in range(0,10,2)]
for index, value in table_B["solve_rate_normalized"].iteritems():
    for upper_bound in bins:
        if value < upper_bound:
            inverse_indexing_rate_B[upper_bound].append(index)
            break

In [ ]:
compare_rate = pd.DataFrame(columns=["table_A_solve_rate_normalized", "table_B_solve_rate_normalized"])
bins = [x/2.0 for x in range(0,20,1)]
count = 0
for table_A_index,value in table_A["solve_rate_normalized"].iteritems():
    if float(count) % round((len(table_A["solve_rate_normalized"])/10.)) == 0:
        print "%.0f%% completed." % (100 * float(count) / float((len(table_A["solve_rate_normalized"]))))
        print str(datetime.now())
    elif float(count) == len(table_A["solve_rate_normalized"])-1:
        print "Finished!"
        print str(datetime.now())
    else:
        pass
    count = count + 1
    bound = 0.0
    for upper_bound in bins:
        if value < upper_bound:
            bound = upper_bound
            break
    for table_B_index in inverse_indexing_rate_B[bound]:
        compare_rate = compare_rate.append([{"table_A_solve_rate_normalized": table_A_index, \
                                   "table_B_solve_rate_normalized": table_B_index}],\
                                 ignore_index=True)

0% completed.
2016-11-12 17:07:02.756835


According to the histogram of problems on solve_rate_normalized, there lies a huge amount of problems in the initial intervals. This causes the blocking to not remove many problem candidates from the set. So this feature is dropped from use in blocking, but this could still be used in matching.